In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time as time
from skyfield.api import load, Topos
from skyfield.framelib import itrs
from astropy.time import Time

In [2]:
# Configuration parameters

Re = 6378     # Mean Earth radius [km]
Hs = 23222    # Approximate satellite orbital altitude [km]
Hion = 1000   # Ionospheric layer maximum altitude [km]

# Orbital period of the satellites [s] (14 h 5 min)
Triv = (14 * 60 * 60) + (5 * 60)

# Angular velocity of the satellite motion [rad/s]
omega = 2 * np.pi / Triv

# Angular resolution of the satellite positions
precision_grad = 0.4                      # Angular resolution in degrees
precision_rad = precision_grad * np.pi / 180  # Angular resolution in radians

# Time step corresponding to the chosen angular resolution [s]
dt = precision_rad / omega

# Time step expressed in days
day_dt = dt / (60 * 60 * 24)

# Number of days considered (used as a scaling or labeling parameter)
DT = 1

# Ionospheric density is assumed constant at a fixed UTC time
# Reference time: 09:00 UTC
instant_h = 9                             # Hour [UTC]
instant_min = instant_h * 60              # Minutes since midnight
instant_sec = instant_min * 60            # Seconds since midnight

# Index of the time step corresponding to the chosen UTC instant
instant_dt = int(instant_sec / dt + 0.5)

# Print the time-step index corresponding to 09:00 UTC
print(instant_dt)


575


In [3]:
def norm1(pos):
    """
    Compute the Euclidean norm of a 3D vector.
    """
    x, y, z = pos
    return np.sqrt(x**2 + y**2 + z**2)


def cartesian(pos, R):
    """
    Convert spherical coordinates (latitude, longitude) to
    Cartesian coordinates on a sphere of radius R.

    Parameters
    ----------
    pos : (lat, lon) in degrees
    R   : sphere radius

    Returns
    -------
    [x, y, z] Cartesian coordinates
    """
    lat, lon = pos
    lat = lat * np.pi / 180
    lon = lon * np.pi / 180

    x = R * np.cos(lat) * np.cos(lon)
    y = R * np.cos(lat) * np.sin(lon)
    z = R * np.sin(lat)

    return [x, y, z]


def spherical_approx(pos):
    """
    Approximate conversion from Cartesian to spherical coordinates.

    Returns latitude, longitude (degrees) and radius.
    """
    x, y, z = pos
    r = norm1(pos)  # Radial distance from Earth's center

    lat = np.arcsin(z / r) * 180 / np.pi   # Latitude in degrees [-90, 90]
    lon = np.arctan2(y, x) * 180 / np.pi   # Longitude in degrees [-180, 180]

    return lat, lon, r


def intersections(pos1, pos2, R=Re + Hion):
    """
    Compute the two intersection points between the line
    defined by pos1–pos2 and a sphere of radius R.

    Returns both intersection points if they exist.
    """
    # Extract Cartesian coordinates
    x1, y1, z1 = pos1
    x2, y2, z2 = pos2
    
    # Direction vector of the line (P2 - P1)
    d = [x2 - x1, y2 - y1, z2 - z1]
    
    # Quadratic equation coefficients: A t^2 + B t + C = 0
    A = d[0]**2 + d[1]**2 + d[2]**2    
    B = 2 * (x1 * d[0] + y1 * d[1] + z1 * d[2])   
    C = x1**2 + y1**2 + z1**2 - R**2
    
    # Discriminant of the quadratic equation
    discriminant = B**2 - 4 * A * C
    
    if discriminant < 0:
        return None  # No intersection with the sphere
    
    sqrt_discriminant = np.sqrt(discriminant)
    t1 = (-B + sqrt_discriminant) / (2 * A)
    t2 = (-B - sqrt_discriminant) / (2 * A)
    
    # Compute intersection points
    intersection1 = [x1 + t1 * d[0], y1 + t1 * d[1], z1 + t1 * d[2]]
    intersection2 = [x1 + t2 * d[0], y1 + t2 * d[1], z1 + t2 * d[2]]
    
    return [intersection1, intersection2]


def intersection(pos1, pos2, R=Re + Hion):
    """
    Compute the physically relevant intersection point between
    the line pos1–pos2 and a sphere of radius R.

    Assumption:
    - One intersection lies beyond pos2 (t > 1)
    - The other lies between pos1 and pos2 (0 < t < 1)
    """
    pos1 = np.array(pos1)
    pos2 = np.array(pos2)

    # Direction vector of the line
    d = pos2 - pos1
    
    # Quadratic equation coefficients
    A = np.dot(d, d)
    B = 2 * np.dot(pos1, d)
    C = np.dot(pos1, pos1) - R**2
    
    discriminant = B**2 - 4 * A * C
    
    if discriminant < 0:
        return None  # No intersection
    
    sqrt_discriminant = np.sqrt(discriminant)
    t1 = (-B + sqrt_discriminant) / (2 * A)
    t2 = (-B - sqrt_discriminant) / (2 * A)

    # Select the closest valid intersection point
    t = np.min([t1, t2])
    intersection = pos1 + t * d

    return intersection



def gamma_def(alpha, Re=Re, Rs=Re + Hion):
    """
    Compute the maximum gamma angle corresponding to a given alpha,
    selecting the physically meaningful solution of the quadratic equation.
    """
    alpha = alpha * np.pi / 180
    cosa2 = np.cos(alpha)**2

    A = Rs**2
    B = -2 * Re * Rs * (1 - cosa2)
    C = Re**2 - (Re**2 + Rs**2) * cosa2

    discriminant = B**2 - 4 * A * C

    cos_gamma1 = (-B - np.sqrt(discriminant)) / (2 * A)
    cos_gamma2 = (-B + np.sqrt(discriminant)) / (2 * A)

    # Select the physically meaningful solution
    cos_gamma = np.max([cos_gamma1, cos_gamma2])
    gamma = 180 * np.arccos(cos_gamma) / np.pi
        
    return gamma


In [4]:
def block_build(total_matrix_pt, total_length_matrix_pt, STEC_pt,
                mes_min, time_st, matrix_number):
    """
    Build block matrices for a tomographic inversion problem.

    The function groups measurements over time into blocks and constructs,
    for each block:
    - the design matrix A
    - the regularization matrix L
    - the observation vector b
    """

    block_matrices = []
    block_b = []

    # Total number of unknowns (voxels)
    num_vars = Nlon * Nlat * Nz

    # Total number of time steps available
    total_steps = len(total_matrix_pt)

    # --- Time blocking strategy ---
    if matrix_number == 0:
        # Compute number of time steps per block based on measurement window
        time_step = int(mes_min / time_st + 0.5)
        print('Number of time steps per matrix:', time_step)

        # Compute number of blocks
        num_blocks = (total_steps + time_step) // time_step
        print('Number of matrices:', num_blocks)
    else:
        # Force a fixed number of blocks
        time_step = (total_steps + matrix_number) // matrix_number
        num_blocks = matrix_number

    # Lists to store block-wise data
    num_eq_total_list = []
    A_blocks = []
    b_blocks = []
    L_blocks = []

    # --- Count equations per block ---
    for block in range(num_blocks):
        start_t = block * time_step
        end_t = min(start_t + time_step, total_steps)

        num_eq_total = 0
        for t in range(start_t, end_t):
            # Each time step may contain a variable number of equations
            num_eq_total += len(total_matrix_pt[t])

        num_eq_total_list.append(num_eq_total)

    print('Number of equations per block:', num_eq_total_list)

    # --- Build matrices for each block ---
    for block in range(num_blocks):
        start_t = block * time_step
        end_t = min(start_t + time_step, total_steps)

        # Initialize dense design matrix A and observation vector b
        A_dense = np.zeros((num_eq_total_list[block], num_vars))
        b_dense = np.zeros(num_eq_total_list[block])

        # Regularization matrix (first-order smoothness constraint)
        L_dense = np.zeros((num_vars - 1, num_vars))

        current_row = 0  # Row counter in A and b

        # Loop over time steps inside the block
        for t in range(start_t, end_t):
            num_eq = len(total_matrix_pt[t])

            for j in range(num_eq):
                # Indices of voxels involved in the equation
                eq_indices = total_matrix_pt[t][j]

                # Corresponding path lengths / coefficients
                eq_coeffs = total_length_matrix_pt[t][j]

                # Fill design matrix row
                for k in range(len(eq_indices)):
                    if eq_indices[k] == -1:
                        # Padding reached: stop reading indices
                        break
                    A_dense[current_row, eq_indices[k]] = eq_coeffs[k]

                # Assign measured STEC value
                b_dense[current_row] = STEC_pt[t][j]

                current_row += 1

        A_blocks.append(A_dense)
        b_blocks.append(b_dense)

        # --- Build regularization matrix L ---
        for k in range(num_vars - 1):
            # Central voxel contribution
            L_dense[k, k] = -3

            # Neighbor in altitude direction
            if k + 1 < num_vars:
                L_dense[k, k + 1] = 1

            # Neighbor in latitude direction
            if k + Nz < num_vars:
                L_dense[k, k + Nz] = 1

            # Neighbor in longitude direction
            if k + (Nz * Nlat) < num_vars:
                L_dense[k, k + (Nz * Nlat)] = 1

        L_blocks.append(L_dense)

    return A_blocks, L_blocks, b_blocks


In [5]:
h = 50      # Voxel height (km)
r = 2       # Voxel horizontal size (longitude/latitude, degrees)

# Maximum elevation angle considered (degrees)
alpha_max = 50

# Central geographic reference point (longitude, latitude)
lon0, lat0 = 10, 50
# lon0, lat0 = 48 - 180, 48  # Alternative reference (commented)
print(f'lon0, lat0 : {lon0, lat0}')

# Additional margin (degrees) to enlarge the region of interest
D = 10

# Maximum angular radius of the ionospheric intersection region
gamma_max = gamma_def(alpha_max, Re, Re + Hion)

# Define minimum and maximum offsets from the central point
# The region is expanded by both the geometric gamma_max and an extra margin D
Dmin = -D - int(gamma_max + r)
Dmax = +D + int(gamma_max + r)

# Longitude and latitude bounds of the voxelized region
lon_min = lon0 + Dmin
lon_max = lon0 + Dmax
lat_min = lat0 + Dmin
lat_max = lat0 + Dmax

print('gamma: ', gamma_max)
print(
    'longitude limits:', lon_min, lon_max,
    'latitude limits:', lat_min, lat_max,
    'region size:', lon_max - lon_min, lat_max - lat_min
)

# Total angular extension of the region
Dln = lon_max - lon_min
Dlt = lat_max - lat_min

# Number of voxels along each spatial direction
Nlon = int(Dln / r)      # Number of voxels in longitude
Nlat = int(Dlt / r)      # Number of voxels in latitude
Nz = int((Hion - h) / h) # Number of voxels in altitude

print('Voxels per edge:', Nlon, Nlat, Nz)
print('Total number of voxels in the region:', Nlon * Nlat * Nz)

# Total number of voxels (flattened 3D grid)
num_voxels = Nlat * Nlon * Nz

lon0, lat0 : (10, 50)
gamma:  8.530883915944756
estremi lon: -10 30 estremi lat: 30 70 lato della regione: 40 40
voxels per spigolo: 20 20 19
numero di voxels nella regione: 7600


In [8]:
# Load pre-saved satellite position data from a pickle file
with open(r"...estrazione dati orbitali e isl\DATI\GALGPS_lowKp_positions_DT=1day_dgrad=0.4.pkl", 'rb') as file: #CHANGE lowKp with highKp to collect the satellite positions on 27 February, during the geomagnetic storm 
    satellite_data = pickle.load(file)

# Create a DataFrame from the loaded dictionary
df = pd.DataFrame(satellite_data)

# Print the number of time steps loaded
print(len(df))

# Show the first few rows of the DataFrame for inspection
df.head()

1534


,Time,GSAT0101 (GALILEO-PFM),GSAT0102 (GALILEO-FM2),GSAT0103 (GALILEO-FM3),GSAT0201 (GALILEO 5),GSAT0202 (GALILEO 6),GSAT0203 (GALILEO 7),GSAT0204 (GALILEO 8),GSAT0205 (GALILEO 9),GSAT0206 (GALILEO 10),...,Sph_GPS BIIF-10 (PRN 08),Sph_GPS BIIF-11 (PRN 10),Sph_GPS BIIF-12 (PRN 32),Sph_GPS BIII-1 (PRN 04),Sph_GPS BIII-2 (PRN 18),Sph_GPS BIII-3 (PRN 23),Sph_GPS BIII-4 (PRN 14),Sph_GPS BIII-5 (PRN 11),Sph_GPS BIII-6 (PRN 28),Sph_GPS BIII-7 (PRN 01)
0,2023-02-25T00:00:00Z,"[-23620.39704247313, -5433.239108153095, -1699...","[-22085.48887788235, -19650.63205356207, 1452....","[5612.995504404883, -18558.718954406973, -2236...","[-29883.934206177873, 6646.907457936617, -197....","[23087.36928728184, 5339.767108357318, -12539....","[11016.43497113807, -11817.233706432464, 24811...","[-19359.559382577892, 2661.331699937419, -2223...","[-17203.56639885161, 2066.76671100383, 23985.4...","[10591.095901000932, 19005.92434883233, -20069...",...,"[-54.72714493236297, 64.06699489486, 26616.987...","[-11.393153057813857, 110.83613621764316, 2630...","[51.899679252270616, 99.05220193191929, 26652....","[20.17283847358287, 0.2468185404499391, 26637....","[-42.01224810031749, -173.0270540331482, 26646...","[-37.41969787580755, 132.90225152683945, 26466...","[-29.527547189143174, -50.72623704762558, 2645...","[17.31039700441655, -121.34209072344915, 26605...","[47.780528975231036, 171.03941864394258, 26556...","[3.5782072982149566, 45.90861766176471, 26568...."
1,2023-02-25T00:00:56Z,"[-23698.55853481542, -5489.1049528886615, -168...","[-22084.86994423054, -19637.76476461922, 1626....","[5731.115437677712, -18603.072625934197, -2230...","[-29859.617836880094, 6644.320911345968, -49.9...","[23033.56723852253, 5390.503195922804, -12688....","[11124.504350661457, -11727.783895729304, 2480...","[-19458.756414331616, 2584.1117972321836, -221...","[-17178.680075069162, 1933.3189417102287, 2401...","[10573.24459691448, 19117.625510897225, -19972...",...,"[-54.77800892787752, 64.63912190840568, 26614....","[-11.787149167450131, 110.8802695271956, 26308...","[51.72903590356044, 99.52090827381416, 26650.6...","[20.54337959721427, 0.3164442460198352, 26637....","[-41.70351360518017, -172.79073692423012, 2664...","[-37.7533306255071, 133.09192984014933, 26467....","[-29.878927092860895, -50.5946806229999, 26459...","[17.687808067789906, -121.28521027754138, 2660...","[47.532341949469824, 171.3982061709331, 26556....","[3.19259318120105, 45.94330322341876, 26568.38..."
2,2023-02-25T00:01:53Z,"[-23776.41985298045, -5544.154799504493, -1673...","[-22083.437937513518, -19624.27917468309, 1799...","[5848.686712601441, -18647.801409854826, -2223...","[-29834.508544978766, 6641.349796442007, 97.55...","[22979.065286897534, 5441.422668344846, -12836...","[11232.952181037434, -11638.849970449566, 2479...","[-19557.964143563946, 2507.6244999256905, -220...","[-17154.342466243575, 1799.6072051513277, 2404...","[10555.971882274978, 19228.861236881465, -1987...",...,"[-54.82350439727277, 65.21330312527073, 26612....","[-12.180890404972173, 110.92521921402339, 2630...","[51.554196607239255, 99.98433431442966, 26649....","[20.913395487408927, 0.3875600135189316, 26637...","[-41.39288669502616, -172.55896876310968, 2664...","[-38.085401922696, 133.28542264739968, 26468.1...","[-30.229266081492785, -50.460552681866865, 264...","[18.064784608745562, -121.22710092265855, 2660...","[47.28111580119657, 171.75136042173756, 26556....","[2.806907231832368, 45.97778681540213, 26568.3..."
3,2023-02-25T00:02:49Z,"[-23853.971808896185, -5598.38939536171, -1660...","[-22081.18785507899, -19610.1824168677, 1972.9...","[5965.702523451589, -18692.899344438036, -2216...","[-29808.606962225163, 6637.986960428521, 245.0...","[22923.873291735046, 5492.530112718511, -12984...","[11341.770742290786, -11550.437837833004, 2479...","[-19657.173383022888, 2431.8722989289945, -219...","[-17130.556488669237, 1665.6402934304351, 2406...","[10539.274445902522, 19339.623231781927, -1977...",...,"[-54.86360

In [10]:
# Load inter-satellite link (ISL) data from a pickle file
with open(
    r"...estrazione dati orbitali e isl\DATI\GALGPS_lowKp_inter_satellite_DT=1day_dgrad=0.4.pkl",
    'rb'
) as file:
    inter_satellite_data = pickle.load(file)

# Create a DataFrame from the loaded ISL data
df_isl = pd.DataFrame(inter_satellite_data)

# Print the number of ISL samples
print(len(df_isl))

# Display the first rows for a quick inspection
df_isl.head()

# Number of sampling points along each inter-satellite link
n_points = 60

# List to store whether each link intersects the selected region
in_out = []

def link_intersects_region(pos1, pos2, lat_min, lat_max, lon_min, lon_max, alt_min, alt_max):
    """
    Check whether the line segment between two satellites intersects
    a given geographic region defined by latitude, longitude, and altitude bounds.
    """
    for t in np.linspace(0, 1, n_points):
        # Compute a point along the inter-satellite link
        x = pos1[0] + t * (pos2[0] - pos1[0])
        y = pos1[1] + t * (pos2[1] - pos1[1])
        z = pos1[2] + t * (pos2[2] - pos1[2])
        
        # Convert Cartesian coordinates to latitude, longitude, and radius
        lat, lon, alt = spherical_approx([x, y, z])
        
        # Check if the point lies inside the selected region
        if lat_min < lat < lat_max and lon_min < lon < lon_max and alt_min < alt < alt_max:
            return True  # The link intersects the region

    return False  # The link does not intersect the region


# Lists to store the intersection points with the ionospheric shell
posi1 = []
posi2 = []

# Compute the intersection points between each ISL and the ionospheric shell
for i in range(len(df_isl)):
    pos1 = df_isl.loc[i, 'r1']  # Position of satellite 1
    pos2 = df_isl.loc[i, 'r2']  # Position of satellite 2

    # Compute the two intersection points with the ionospheric sphere
    pi1, pi2 = intersections(pos1, pos2, Re + Hion)

    posi1.append(pi1)
    posi2.append(pi2)

# Store ionospheric intersection points in the DataFrame
df_isl['ri1'] = posi1
df_isl['ri2'] = posi2


# Check whether the ionospheric segment of each link crosses the selected region
for i in range(len(df_isl)):
    pos1 = df_isl.loc[i, 'ri1']
    pos2 = df_isl.loc[i, 'ri2']

    in_out.append(
        link_intersects_region(
            pos1, pos2,
            lat_min, lat_max,
            lon_min, lon_max,
            Re + h, Re + Hion
        )
    )

# Add a boolean flag indicating whether the ISL is useful for the selected region
df_isl['in_out'] = in_out

# Select only the ISLs intersecting the region of interest
util_isl = df_isl[df_isl['in_out']].copy().reset_index()

print(f'Number of ISLs crossing the selected region: {len(util_isl)}')


42797
numero di ISL nella regione scelta: 2757


In [ ]:
def vis_alpha(pos1, pos2):
    # Satellite position (pos1) and reference point (pos2, typically Earth center projection)
    xs, ys, zs = pos1[0], pos1[1], pos1[2]
    x0, y0, z0 = pos2[0], pos2[1], pos2[2] 

    # Compute cosine of the visibility angle alpha
    # Based on the scalar product between (Ps - P0) and P0
    # |Ps - P0| * |P0| * cos(alpha) = (Ps - P0) · P0
    cosalpha = ((xs - x0) * x0 + (ys - y0) * y0 + (zs - z0) * z0) / (norm1(pos2 - pos1) * Re)
    
    return cosalpha

def lat_intersec(lat, r1, r2):
    """
    Compute the intersection points between a segment (r1, r2)
    and a constant-latitude conical surface.
    """
    times = []
    t1 = -1
    t2 = -1
    delta = r2 - r1
    tan2 = np.tan(np.radians(lat)) ** 2
    ri = []

    # Quadratic equation coefficients
    A = delta[2]**2 - tan2 * (delta[0]**2 + delta[1]**2)
    B = 2 * (r1[2] * delta[2] - tan2 * (r1[0] * delta[0] + r1[1] * delta[1]))
    C = r1[2]**2 - tan2 * (r1[0]**2 + r1[1]**2)

    discriminant = B**2 - 4 * A * C

    # Only real intersections are physically meaningful
    if discriminant > 0:
        t1 = (-B + np.sqrt(discriminant)) / (2 * A)
        t2 = (-B - np.sqrt(discriminant)) / (2 * A)

    # Keep only intersections lying on the segment
    for t in [t1, t2]:
        if 0 <= t <= 1:
            ri.append(r1 + t * delta)
            times.append(t)

    return ri, times

def lon_intersec(lon, r1, r2):
    """
    Compute the intersection between a segment and a constant-longitude plane.
    """
    delta = r2 - r1
    times = []
    ri = []

    tan_lon = np.tan(lon * (np.pi / 180))
    t = (tan_lon * r1[0] - r1[1]) / (delta[1] - tan_lon * delta[0])

    # Accept only intersections within the segment
    if 0 <= t <= 1:
        ri.append(r1 + t * delta)
        times.append(t)

    return ri, times


def intersection_and_t(closest_point, far_point, alt):
    """
    Compute intersections between a segment and a spherical shell
    of radius 'alt'. Also return the corresponding parametric t.
    """
    pos1 = np.array(closest_point)
    pos2 = np.array(far_point)
    R = alt

    d = pos2 - pos1

    # Quadratic equation coefficients
    A = np.dot(d, d)
    B = 2 * np.dot(pos1, d)
    C = np.dot(pos1, pos1) - R**2

    discriminant = B**2 - 4 * A * C

    if discriminant < 0:
        return [None, None]  # No intersection

    sqrt_discriminant = np.sqrt(discriminant)
    t1 = (-B + sqrt_discriminant) / (2 * A)
    t2 = (-B - sqrt_discriminant) / (2 * A)

    ri = []
    ti = []

    # Keep only valid intersections on the segment
    for t in [t1, t2]:
        if 0 <= t <= 1:
            ri.append(pos1 + t * d)
            ti.append(t)

    return ri, ti


def find_orizontal_voxel_intersections(closest_point, far_point, lat_grid, lon_grid, alt_grid):
    """
    Find all intersection points between a segment (closest_point, far_point)
    and a voxel grid defined by latitude, longitude, and altitude boundaries.

    closest_point: entry point of the ray into the region.
    far_point: exit point of the ray from the region.
    lat_grid: array of latitude boundaries defining voxel faces.
    lon_grid: array of longitude boundaries defining voxel faces.
    alt_grid: array of altitude boundaries defining voxel faces.

    Returns an ordered list of intersection points along the ray.
    """
    abs_tol = 1e-6
    rel_tol = 1e-6

    # Characteristic length of the segment, used to define a numerical tolerance
    L = np.linalg.norm(far_point - closest_point)
    eps = max(abs_tol, rel_tol * (L if L > 0 else 1.0))

    ri, t = [], []

    # Add starting point of the segment
    ri.append(P1)
    t.append(0.0)

    delta = np.array(far_point - closest_point)

    # Intersections with latitude surfaces
    for lat in lat_grid:
        riapp, tapp = lat_intersec(lat, closest_point, far_point)
        ri.extend(riapp)
        t.extend(tapp)

    # Intersections with longitude surfaces
    for lon in lon_grid:
        riapp, tapp = lon_intersec(lon, closest_point, far_point)
        ri.extend(riapp)
        t.extend(tapp)

    # Intersections with altitude (spherical) surfaces
    for alt in alt_grid:
        riapp, tapp = intersection_and_t(closest_point, far_point, alt)
        if riapp is not None:
            for i, inter in enumerate(riapp):
                ri.append(inter)
                t.append(tapp[i])

    # Add final point of the segment
    ri.append(P2)
    t.append(1.0)

    # Sort all candidate intersection points along the ray
    candidates = list(zip(t, ri))
    candidates.sort(key=lambda it: it[0])

    # Remove duplicated intersection points caused by numerical or geometric symmetry
    uniq = []
    for tval, p in candidates:
        if not uniq:
            uniq.append((tval, p))
            continue

        last_t, last_p = uniq[-1]
        if np.linalg.norm(p - last_p) <= eps:
            # Points are effectively identical: keep the first one
            continue
        else:
            uniq.append((tval, p))

    # Return only the ordered, de-duplicated intersection points
    ordered_points = [p for (t, p) in uniq]
    return ordered_points


    


def ray_error_orizontal_voxel_intersections(P1, P2, Nx1, Nx2, Nx3, r, h,
                                            initial_grid, density,
                                            voxels, lengths,
                                            matrix_idx, original_idx_inv,
                                            estec):
    """
    Computes the Slant Total Electron Content (STEC) along an horizontal
    inter-satellite ray, accounting for voxel intersections and adding
    Gaussian measurement noise.

    P1, P2            : Cartesian coordinates of the ray endpoints
    Nx1, Nx2, Nx3     : Number of voxels along longitude, latitude, altitude
    r, h              : Horizontal (deg) and vertical (km) voxel sizes
    initial_grid      : [lat_min, lon_min, alt_min] of the voxel grid
    density           : Electron density values for each voxel
    voxels, lengths   : Output arrays storing crossed voxel indices and path lengths
    matrix_idx        : Index of the current block matrix
    original_idx_inv  : Mapping from reduced to original voxel indices
    estec             : Standard deviation of the STEC measurement error
    """

    # Number of voxels in a latitude–altitude slice
    Nx23 = Nx2 * Nx3

    # Small offset used to move slightly inside the voxel (numerical stability)
    alpha = 0.001

    k = 0            # Counter for crossed voxels
    stec = 0.0       # Accumulated STEC along the ray

    # Compute intersection points between the ray and the voxel grid
    rinters = find_orizontal_voxel_intersections(P1, P2, lat_grid, lon_grid, alt_grid)

    # Starting point of the ray inside the grid
    r0 = rinters[0]
    sh0 = spherical_approx(r0)

    # Loop over consecutive intersection points
    for i, ri in enumerate(rinters[1:]):

        shi = spherical_approx(ri)

        # Direction vector between consecutive intersection points
        dir_vec = ri - r0
        d = np.linalg.norm(dir_vec)

        # Midpoint slightly shifted inside the segment to avoid boundary ambiguity
        if d > 0:
            rm = r0 + alpha * dir_vec
        else:
            rm = r0.copy()

        shm = spherical_approx(rm)

        # Compute voxel indices (lon, lat, alt) from spherical coordinates
        x1 = int(np.floor((shm[1] - initial_grid[1]) / r))      # Longitude index
        x2 = int(np.floor((shm[0] - initial_grid[0]) / r))      # Latitude index
        x3 = int(np.floor((shm[2] - Re - h) / h))               # Altitude index

        # Linear voxel index
        vnew = x1 * Nx23 + x2 * Nx3 + x3

        # Check if the voxel belongs to the reduced system matrix
        if vnew in original_idx_inv[matrix_idx]:

            # Debug check for repeated voxel intersections
            if (k != 0) and (vold == vnew):
                print(vnew)
                print(shm[0], shm[1], (shm[2] - Re - h) / h)
                print('new_length:', norm1(ri - r0))
                print('old_length:', lengths[k - 1] / 1000)
                print()

            # Store path length inside the voxel [m]
            lengths[k] = norm1(ri - r0) * 1000

            # Store voxel index
            voxels[k] = vnew

            # Accumulate STEC contribution
            stec += density[vnew] * lengths[k]

            k += 1
            vold = vnew

        # Update starting point for next segment
        r0 = ri
        sh0 = shi

    # Convert STEC to TECU and add Gaussian measurement noise
    stec_measured = stec * 1e-16 + np.random.normal(loc=0.0, scale=estec)

    return stec_measured


In [12]:

def find_vertical_voxel_intersections(closest_point, far_point, lat_grid, lon_grid, alt_grid):
    """
    Find all intersection points between a ray segment and
    a vertical voxel grid defined by latitude, longitude, and altitude planes.
    """
    rel_tol = 1e-6
    L = np.linalg.norm(far_point - closest_point)
    eps = rel_tol * (L if L > 0 else 1.0)

    ri, t = [], []
    ri.append(closest_point)
    t.append(0)

    # Latitude planes
    for lat in lat_grid:
        riapp, tapp = lat_intersec(lat, closest_point, far_point)
        ri.extend(riapp)
        t.extend(tapp)

    # Longitude planes
    for lon in lon_grid:
        riapp, tapp = lon_intersec(lon, closest_point, far_point)
        ri.extend(riapp)
        t.extend(tapp)

    # Altitude (spherical shells)
    for alt in alt_grid:
        riapp, tapp = intersection_and_t(closest_point, far_point, alt)
        if riapp is not None:
            for i, inter in enumerate(riapp):
                ri.append(inter)
                t.append(tapp[i])

    # Add final point of the segment
    ri.append(P2)
    t.append(1.0)

    # Sort intersection points along the ray
    candidates = list(zip(t, ri))
    candidates.sort(key=lambda it: it[0])

    # Remove duplicate intersections caused by symmetry
    uniq = []
    for tval, p in candidates:
        if not uniq:
            uniq.append((tval, p))
            continue

        last_t, last_p = uniq[-1]
        if np.linalg.norm(p - last_p) <= eps:
            continue
        else:
            uniq.append((tval, p))

    ordered_points = [p for (t, p) in uniq]
    return ordered_points

In [14]:
def find_vertical_voxel_intersections(closest_point, far_point, lat_grid, lon_grid, alt_grid):
    """
    Trova i punti di intersezione tra il segmento (closest_point, far_point) e la griglia di voxel.
    
    closest_point: punto più vicino che segna l'entrata nella regione.
    far_point: punto più lontano rispetto a closest_point.
    lat_grid: array delle latitudini che definiscono i voxel.
    lon_grid: array delle longitudini che definiscono i voxel.
    alt_grid: array delle altitudini che definiscono i voxel.
    
    Restituisce una lista ordinata di punti di intersezione.
    """
    rel_tol = 1e-6
    L = np.linalg.norm(far_point - closest_point)
    eps = rel_tol * (L if L > 0 else 1.0)
    
    ri, t = [], []
    ri.append(closest_point)
    t.append(0)
    delta = np.array(far_point - closest_point)
    # Trova i valori di t per i piani di latitudine
    for lat in lat_grid:
        riapp, tapp = lat_intersec(lat, closest_point, far_point)
        ri.extend(riapp)  
        t.extend(tapp)  
        
    # Trova i valori di t per i piani di longitudine
    for lon in lon_grid:
        riapp, tapp = lon_intersec(lon, closest_point, far_point)
        ri.extend(riapp)  
        t.extend(tapp)

    for alt in alt_grid:
        riapp, tapp = intersection_and_t(closest_point, far_point, alt)
        if riapp is not None:
            for i, inter in enumerate(riapp):
                ri.append(inter)
                t.append(tapp[i])

    # Ordina i punti di intersezione lungo il segmento
    # PER RAGIONI DI SIMMETRIA PUò ACCADERE CHE LA RETTA INTERSECHI DUE SUPERFICI NELLO STESSO PUNTO (I DUE CONI + LAT, -LAT O I DUE PIANI +LON, -LON)
    # CREANDO DEI DUPLICATI IN ri, ti, CHE ORA TOLGO
    ri.append(P2)
    t.append(1.0)
    
    candidates = list(zip(t, ri))
    candidates.sort(key=lambda it: it[0])

    uniq = []
    for tval, p in candidates:
        if not uniq:
            uniq.append((tval, p))
            continue
        last_t, last_p = uniq[-1]
        if np.linalg.norm(p - last_p) <= eps:
            # sono praticamente lo stesso punto: tieni quello con t più piccolo (già ordinati),
            # o aggiorna t a media se vuoi. Qui manteniamo il primo (last).
            # In alternativa potresti sostituire last_p con media: last_p = 0.5*(last_p+p)
            continue
        else:
            uniq.append((tval, p))

    # ora ritorna solo i punti (ordinati e de-duplicati)
    ordered_points = [p for (t,p) in uniq]
    return ordered_points
    


def ray_error_vertical_voxel_intersections(P1, P2, Nx1, Nx2, Nx3, r, h,
                                          initial_grid, density,
                                          voxels, lengths, estec):
    """
    Computes the Slant Total Electron Content (STEC) along a vertical
    satellite–ground ray, accounting for voxel intersections and adding
    Gaussian measurement noise.

    P1, P2        : Cartesian coordinates of the ray entry and exit points
    Nx1, Nx2, Nx3 : Number of voxels along longitude, latitude, altitude
    r, h          : Horizontal (deg) and vertical (km) voxel sizes
    initial_grid  : [lat_min, lon_min, alt_min] of the voxel grid
    density       : Electron density values for each voxel
    voxels        : Output array storing crossed voxel indices
    lengths       : Output array storing path lengths inside each voxel [m]
    estec         : Standard deviation of the STEC measurement error
    """

    # Number of voxels in a latitude–altitude slice
    Nx23 = Nx2 * Nx3

    k = 0                 # Counter for crossed voxels
    stec = 0.0            # Accumulated STEC along the ray
    num_variables = Nx1 * Nx23  # Total number of voxels in the grid

    # Ray endpoints
    r0 = P1
    rf = P2

    # Small fraction of the segment used to move slightly inside the voxel
    # (avoids numerical issues at voxel boundaries)
    alpha = 1e-3

    # Compute intersection points between the ray and the voxel grid
    rinters = find_vertical_voxel_intersections(r0, rf, lat_grid, lon_grid, alt_grid)
    sh0 = spherical_approx(r0)

    # Loop over consecutive intersection points
    for i, ri in enumerate(rinters[1:]):

        shi = spherical_approx(ri)

        # Direction vector between consecutive intersection points
        dir_vec = ri - r0
        d = np.linalg.norm(dir_vec)

        # Midpoint slightly shifted inside the segment
        if d > 0:
            rm = r0 + alpha * dir_vec
        else:
            rm = r0.copy()

        shm = spherical_approx(rm)

        # Check if the midpoint lies inside the region of interest
        if ((lat_min <= shm[0] <= lat_max) and
            (lon_min <= shm[1] <= lon_max) and
            (Re + h <= shm[2] <= Re + Hion)):

            # Compute voxel indices (lon, lat, alt)
            x1 = int(np.floor((shm[1] - initial_grid[1]) / r))      # Longitude index
            x2 = int(np.floor((shm[0] - initial_grid[0]) / r))      # Latitude index
            x3 = int(np.floor((shm[2] - Re - h) / h))               # Altitude index

            # Linear voxel index
            vnew = x1 * Nx23 + x2 * Nx3 + x3

            # Check that the voxel index is valid
            if vnew < num_variables:

                # Debug check for repeated voxel intersections
                if (k != 0) and (vold == vnew):
                    print('vnew, vold:', vnew)
                    print('intersection index:', i, '/', len(rinters[1:]))
                    sh00 = spherical_approx(rinters[i - 2])
                    print('problematic points:')
                    print(sh00[0], sh00[1], (sh00[2] - Re - h) / h)
                    print(sh0[0], sh0[1], (sh0[2] - Re - h) / h)
                    print(x1, x2, x3)
                    print(shm[0], shm[1], (shm[2] - Re - h) / h)
                    print(shi[0], shi[1], (shi[2] - Re - h) / h)
                    print('dir_vec:', dir_vec)
                    print('segment length:', d)
                    print('new_length:', norm1(ri - r0))
                    print('old_length:', lengths[k - 1] / 1000)
                    print()

                # Store path length inside the voxel [m]
                lengths[k] = norm1(ri - r0) * 1000

                # Store voxel index
                voxels[k] = vnew

                # Accumulate STEC contribution
                stec += density[vnew] * lengths[k]

                k += 1
                vold = vnew

        # Update starting point for the next segment
        r0 = ri
        sh0 = shi

    # Convert STEC to TECU and add Gaussian measurement noise
    stec_measured = stec * 1e-16 + np.random.normal(loc=0.0, scale=estec)

    return stec_measured


In [15]:
def total_equations(mv, mo, tv, to, time_window):
    """
    Merge vertical and horizontal equation sets over a given time window.

    mv: list of equation blocks for vertical rays
    mo: list of equation blocks for horizontal rays
    tv: temporal sampling interval for vertical equations
    to: temporal sampling interval for horizontal equations
    time_window: total number of discrete time steps

    Returns a list where each element contains all equations
    active at a given time step.
    """
    # mv and mo may have different time resolutions; t1 and t2
    # track the current index in the vertical and horizontal lists
    total_eq_pt = []
    t1 = 0
    t2 = 0

    # Loop over the full time window
    for t in range(time_window):
        total_eq = []

        # Add vertical equations at their sampling times
        if t % tv == 0:
            for eq in mv[t1]:
                total_eq.append(eq)
            t1 += 1

        # Add horizontal equations at their sampling times
        if t % to == 0:
            for eq in mo[t2]:
                total_eq.append(eq)
            t2 += 1

        # Store all equations active at time t
        total_eq_pt.append(total_eq)

    return total_eq_pt


def total_b(bv, bo, tv, to, time_window):
    """
    Merge vertical and horizontal observation vectors (right-hand side)
    over a given time window.

    bv: list of observation blocks for vertical rays
    bo: list of observation blocks for horizontal rays
    tv: temporal sampling interval for vertical observations
    to: temporal sampling interval for horizontal observations
    time_window: total number of discrete time steps

    Returns a list where each element contains all observation values
    available at a given time step.
    """
    total_b_pt = []
    t1 = 0
    t2 = 0

    # Loop over the full time window
    for t in range(time_window):
        total_b = []

        # Add vertical observations at their sampling times
        if t % tv == 0:
            for b in bv[t1]:
                total_b.append(b)
            t1 += 1

        # Add horizontal observations at their sampling times
        if t % to == 0:
            for b in bo[t2]:
                total_b.append(b)
            t2 += 1

        # Store all observations available at time t
        total_b_pt.append(total_b)

    return total_b_pt


In [16]:
# Print a description of the temporal resolution assumptions:
# the time step is derived from a fixed angular displacement (precision_grad)
# assuming a circular orbit and constant orbital speed
print(f'''The temporal resolution is computed from a variation of {precision_grad} degrees
of a satellite, assuming a spherical orbit and constant angular velocity''')

# Print the satellite angular frequency (rad/s)
print(f'Satellite angular frequency (omega): {omega} 1/s')

# Print the temporal resolution in seconds and in days
print(f'Temporal resolution: {dt} seconds, equivalent to {day_dt} days')


# GNSS carrier frequencies (in Hz)
# L1, L2, and L5 correspond to standard GNSS signal bands
L1 = 154 * 10.23 * 1e6
L2 = 120 * 10.23 * 1e6
L5 = 1176.450 * 1e6 

# Assumed ionospheric delay error (TECU)
I = 1

# STEC error estimation for each frequency band
# Formula derived from ionospheric delay theory
estec1 = L1**2 / (40.3 * 1e16) * I
estec2 = L2**2 / (40.3 * 1e16) * I
estec5 = L5**2 / (40.3 * 1e16) * I

# Print frequencies
print(f'L1: {L1}, L2: {L2}, L5: {L5}')

# Print estimated STEC errors for each GNSS frequency
print(f'STEC error assuming ionospheric error I={I}: '
      f'L1: {estec1}, L2: {estec2}, L5: {estec5}')


La precisione temporale è calcolata in base alla variazione di 0.4 gradi di un satellite, 
ammettendo un'orbita sferica e velocità costante in modulo
frequenza (omega) in Hz dei satelliti: 0.00012392870428362104 1/s
La precisione temporale: 56.33333333333333 secondi, in giorni: 0.0006520061728395062
L1: 1575420000.0, L2: 1227600000.0000002, L3: 1176450000.0
Errore stec dato I:errore iono 1 con L1: 6.158680338461538, L2: 3.739458461538463, L5: 3.4343290384615384


In [17]:
# Fractional reduction of ionospheric error for inter-satellite links (ISL)
isl_iono_reduction = 0.10

# Example ionospheric error value (TECU)
Iiono = np.array([0.15])

# List of STEC measurement errors
# list_estec = np.array([estec5])  # Could use L5-based error
list_estec = np.array([1])         # Using arbitrary normalized value here
list_estec_oriz = []               # To store horizontal link STEC errors

# Convert measurement intervals in minutes to number of time steps
# list_mes_min = [30, 40, 50, 60, 75, 90] minutes converted to seconds / dt
list_mes_min = np.array([30, 40, 50, 60, 75, 90]) * 60 / dt

# Compute horizontal link STEC error (reduced fraction)
for estec in list_estec:
    list_estec_oriz.append(estec * isl_iono_reduction) 

# Number of ground stations considered
Nstations = np.array([64])
print(Nstations)

# Approximate distance between stations, assuming uniform grid
# ds = sqrt(area per station)
ds = np.sqrt(4 * D/2 * D/2 / Nstations)  # D/2 squared area per quadrant
for i in range(len(Nstations)):
    print(ds[i], Nstations[i])

# Print original list of STEC errors
print(list_estec)
estec = list_estec[0]  # Use first element as reference
print()
print(estec)
print()


[64]
1.25 64
[1]

1



**TEST CYCLE FOR STEC COLLECTION**

In [18]:
# ===============================================
# 1. Define the voxel grid
# ===============================================
lat_grid = np.arange(lat_min, lat_max, r)        # Latitude grid for voxels (step of r degrees)
lon_grid = np.arange(lon_min, lon_max, r)        # Longitude grid for voxels (step of r degrees)
alt_grid = np.arange(Re + (2*h), Re + Hion, h)  # Altitude grid for voxels (from lower ionosphere to Hion)

# Minimum point of the grid (latitude, longitude, altitude)
grid_min = np.array([lat_min, lon_min, Re+h])

# ===============================================
# 2. Load the ionospheric density
# ===============================================
density = np.load(
    r"C...estrazione dati orbitali e isl\DATI\flatten_iri_stormionosphere.npy"
)

# Print the dimensions of the voxel grid
print(f'Voxel grid dimensions: {Nlon, Nlat, Nz}')

# ===============================================
# 3. Discretization parameters
# ===============================================
num_vertical_steps = int((Hion-h)/h)  # Number of vertical steps in the voxel grid
num_steps = 40  # Total number of steps (can include horizontal steps)
tim = time.time()  # Current timestamp for timing purposes

grid_min = np.array([lat_min, lon_min, Re+h])  # Minimum point of the grid (redundant here, ensures consistency)

# ===============================================
# 4. Define terrestrial station grid
# ===============================================
d = 10  # Grid spacing for ground stations in degrees
print('Station grid spacing:', d)

P0 = []  # List to store ground station positions
D = 20   # Half-width of the region to place stations

# Loop over grid offsets to place stations in the region
for d0 in np.arange(d/2, int(D), d):
    for d1 in np.arange(d/2, int(D), d):
        # Append positions in lat/lon coordinates
        P0.append([lat_min + d0 + D/2, lon_min + D/2 + d1])

print(P0)

# Convert ground station positions from geodetic to Cartesian coordinates
for i in range(len(P0)):
    P0[i] = np.array(cartesian(P0[i], Re))

print('Number of ground stations placed:', len(P0),
      'with grid spacing:', d,
      'in a region of size:', D*2, 'x', D*2,
      'within a cone of', Dln, 'x', Dlt)

# ===============================================
# 5. Initialize arrays to store measurements
# ===============================================
STEC_pt = []               # Slant Total Electron Content (STEC) per time step
total_matrix_pt = []       # Voxel indices for each vertical link
total_length_matrix_pt = []  # Path lengths through each voxel
satelliti_visti = 0        # Counter for visible satellites
visib_voxels_pertime = []  # Temporary storage per time step

time_stv = 2               # Sampling interval for vertical links (in time steps)

# ===============================================
# 6. Maximum elevation angle cosine for visibility
# ===============================================
cosalpha_max = np.cos(alpha_max * np.pi / 180)  # Cosine of max elevation angle
print('################', cosalpha_max)

# ===============================================
# 7. Measurement interval in steps
# ===============================================
mes_min = int(30 * 60 / dt + 0.5)  # Number of steps in 30 minutes

# ===============================================
# 8. Initialize arrays for inter-satellite links
# ===============================================
STEC_isl_pt = []             # STEC for inter-satellite links
total_matrix_isl_pt = []     # Voxel indices for ISL
total_length_matrix_isl_pt = []  # Path lengths for ISL
STEC_pt = []                 # Reset STEC arrays
total_matrix_pt = []         # Reset voxel matrices
total_length_matrix_pt = []  # Reset path lengths


# ==========================================================
# COUNT VERTICAL LINKS
# ==========================================================
for u, estec in enumerate(list_estec):

    print()
    print()
    print('###############################################################')
    print()
    print('time window block:', mes_min)
    print()

    # ----------------------------------------------------------
    # Loop over time steps inside the time window
    # ----------------------------------------------------------
    for t in range(instant_dt, instant_dt + mes_min):

        # Progress print every 10 steps
        if t % 10 == 0:
            print(f"Iteration t: {t - instant_dt} / {mes_min}")

        satelliti_visti = 0  # Counter for visible satellites

        STEC = []                 # STEC measurements at this time step
        total_matrix = []         # Voxel indices for each equation
        total_length_matrix = []  # Path lengths through each voxel

        # Use only one vertical equation every time_stv steps
        # to reduce correlation between equations
        if t % time_stv == 0:

            # Loop over ground stations
            for id_pos, pos0 in enumerate(P0):

                # Loop over satellites
                for id_sat, satellite in enumerate(satellite_names):

                    # Satellite position at time t
                    pos1 = np.array(df.loc[t, satellite])

                    # Compute cosine of elevation angle
                    cosalpha = vis_alpha(pos1, pos0)

                    if cosalpha > cosalpha_max:

                        # Initialize voxel index and length arrays
                        voxels_vert_mx = -np.ones(int(num_steps), dtype=int)
                        lengths_vert_mx = -np.ones(int(num_steps))

                        satelliti_visti += 1

                        # Compute intersection with lower and upper ionosphere boundaries
                        xi1, yi1, zi1 = intersection(pos1, pos0, R=Re + h)
                        xi2, yi2, zi2 = intersection(pos1, pos0, R=Re + Hion)

                        P1 = np.array([xi1, yi1, zi1])  # Entry point
                        P2 = np.array([xi2, yi2, zi2])  # Exit point

                        # Compute STEC with noise along vertical ray
                        stec_vert = ray_error_vertical_voxel_intersections(
                            P1, P2, Nlon, Nlat, Nz, r, h,
                            grid_min, density,
                            voxels_vert_mx, lengths_vert_mx,
                            estec
                        )

                        # Store ground station and satellite IDs
                        voxels_vert_mx[-2] = id_pos
                        lengths_vert_mx[-2] = id_pos

                        voxels_vert_mx[-1] = id_sat
                        lengths_vert_mx[-1] = id_sat

                        # Store equation
                        total_matrix.append(voxels_vert_mx)
                        total_length_matrix.append(lengths_vert_mx)
                        STEC.append(stec_vert)

        # Store time step data
        STEC_pt.append(STEC)
        total_matrix_pt.append(total_matrix)
        total_length_matrix_pt.append(total_length_matrix)

    print('Total time steps:', len(total_matrix_pt))

    # ==========================================================
    # BUILD BLOCK MATRICES FOR VERTICAL LINKS
    # ==========================================================
    A_blocks, L_blocks, b_blocks = block_build(
        total_matrix_pt,
        total_length_matrix_pt,
        STEC_pt,
        mes_min,
        1,
        0
    )

    fil_A_blocks = []   # Filtered A matrices
    fil_L_blocks = []   # Filtered regularization matrices
    non_zero_columns = []

    # Remove columns corresponding to unused voxels
    for i in range(len(A_blocks)):
        non_zero_columns.append(np.any(A_blocks[i] != 0, axis=0))

        filtered_A = A_blocks[i][:, non_zero_columns[-1]]
        filtered_L = L_blocks[i][non_zero_columns[-1][:-1]][:, non_zero_columns[-1]]

        fil_A_blocks.append(filtered_A)
        fil_L_blocks.append(filtered_L)

        print(i, 'A_block', 'old shape:', A_blocks[i].shape,
              'new shape:', filtered_A.shape)

    # ----------------------------------------------------------
    # Mapping between reduced and original voxel indices
    # ----------------------------------------------------------
    original_idx = []
    original_idx_inv = []

    for k in range(len(non_zero_columns)):
        original_idx.append({
            j: i for j, i in enumerate(
                [i for i in range(len(non_zero_columns[k])) if non_zero_columns[k][i]]
            )
        })
        original_idx_inv.append({
            i: j for j, i in enumerate(
                [i for i in range(len(non_zero_columns[k])) if non_zero_columns[k][i]]
            )
        })

    # ==========================================================
    # HORIZONTAL (INTER-SATELLITE) LINKS
    # ==========================================================
    time_sto = 1
    num_steps = 80
    matrix_idx = 0

    print('horizontal merging:')
    time_step = int(mes_min / time_sto)
    print(time_step)

    for t in range(instant_dt, instant_dt + mes_min):

        STEC = []
        total_matrix = []
        total_length_matrix = []

        # Extract ISL positions at time t
        pos_s = util_isl[util_isl['time'] == t][['ri1', 'ri2']].reset_index()

        for row in range(len(pos_s)):

            voxels_oriz_mx = -np.ones(int(num_steps), dtype=int)
            lengths_oriz_mx = -np.ones(int(num_steps))

            P1 = np.array(pos_s.loc[row, 'ri1'])
            P2 = np.array(pos_s.loc[row, 'ri2'])

            # Compute STEC along horizontal ray with reduced ionospheric error
            stec_oriz = ray_error_orizontal_voxel_intersections(
                P1, P2, Nlon, Nlat, Nz, r, h,
                grid_min, density,
                voxels_oriz_mx, lengths_oriz_mx,
                matrix_idx, original_idx_inv,
                estec * isl_iono_reduction
            )

            if voxels_oriz_mx[0] > -0.5:
                total_matrix.append(voxels_oriz_mx)
                total_length_matrix.append(lengths_oriz_mx)
                STEC.append(stec_oriz)

        STEC_isl_pt.append(STEC)
        total_matrix_isl_pt.append(total_matrix)
        total_length_matrix_isl_pt.append(total_length_matrix)

    # ==========================================================
    # BUILD BLOCK MATRICES FOR ISL AND MERGED SYSTEM
    # ==========================================================
    Aisl_blocks, Lisl_blocks, bisl_blocks = block_build(
        total_matrix_isl_pt,
        total_length_matrix_isl_pt,
        STEC_isl_pt,
        mes_min,
        1,
        0
    )

    # Merge vertical and horizontal equations
    total_matrix_hv_pt = total_equations(
        total_matrix_pt,
        total_matrix_isl_pt,
        1,
        1,
        mes_min
    )

    total_length_matrix_hv_pt = total_equations(
        total_length_matrix_pt,
        total_length_matrix_isl_pt,
        1,
        1,
        mes_min
    )

    STEC_hv_pt = total_b(
        STEC_pt,
        STEC_isl_pt,
        1,
        1,
        mes_min
    )

    Ahv_blocks, Lhv_blocks, bhv_blocks = block_build(
        total_matrix_hv_pt,
        total_length_matrix_hv_pt,
        STEC_hv_pt,
        mes_min,
        1,
        0
    )

    # Filter merged matrices
    fil_Ahv_blocks = []

    for i in range(len(A_blocks)):
        non_zero_columns_hv = np.any(Ahv_blocks[i] != 0, axis=0)
        filtered_A = Ahv_blocks[i][:, non_zero_columns_hv]
        fil_Ahv_blocks.append(filtered_A)

        print(i, 'A_block', 'old shape:', Ahv_blocks[i].shape,
              'new shape:', filtered_A.shape)

    # Consistency checks
    for i in range(len(Ahv_blocks)):
        if len(Ahv_blocks[i]) - len(A_blocks[i]) != len(Aisl_blocks[i]):
            print(f'Wrong allocation of horizontal equations in block {i}')

        if np.shape(fil_Ahv_blocks[i])[1] != np.shape(fil_A_blocks[i])[1]:
            print(f'COLUMN MISMATCH IN BLOCK {i}:',
                  np.shape(fil_Ahv_blocks[i])[1],
                  'vs',
                  np.shape(fil_A_blocks[i])[1])


dimensioni griglia voxels:(20, 20, 19)
precisione griglia stazioni: 10
[[45.0, 5.0], [45.0, 15.0], [55.0, 5.0], [55.0, 15.0]]
Numero di stazioni terrestri poste: 4 su una griglia di passo: 10 in una regione 40 x 40 in un cono 40 x 40
################ 0.6427876096865394


###############################################################

time window block: 32

Iterazione t:5 / 32
Iterazione t:15 / 32
Iterazione t:25 / 32
tot istanti temporali 32
numero di istanti temporali per matrice: 32
numero di matrici: 2
numero di equazioni per ogni blocco: [443, 0]
0 A_block old shape: (443, 7600) new shape: (443, 1008)
1 A_block old shape: (0, 7600) new shape: (0, 0)
horizzontal merging:
32
numero di istanti temporali per matrice: 32
numero di matrici: 2
numero di equazioni per ogni blocco: [44, 0]
entrata in total_equtions: 32 32 dimensioni di sistemi sparti verticali e orizzontali
entrata in total_equtions: 32 32 dimensioni di sistemi sparti verticali e orizzontali
numero di istanti temporali per

In [20]:
import plotly.graph_objects as go

def decode_id_lonlat(ID):
    """
    Decode a voxel linear ID into physical longitude, latitude and altitude.

    ID: linear voxel index
    Returns:
        lon [deg], lat [deg], alt [km]
    """
    alt = ID % Nz * h
    lat = ((ID // Nz) % Nlat + (grid_min[0] / r)) * r
    lon = (ID // (Nlat * Nz) + (grid_min[1] / r)) * r
    return (lon, lat, alt)


def decode_id(ID):
    """
    Decode a voxel linear ID into voxel grid indices (lon_idx, lat_idx, alt_idx).

    ID: linear voxel index
    Returns:
        (lon_index, lat_index, alt_index)
    """
    alt = ID % Nz
    lat = ((ID // Nz) % Nlat)
    lon = (ID // (Nlat * Nz) % Nlon)
    return (lon, lat, alt)

# Select the block index to be visualized
idx = 0

# Filtered system matrix for the selected block
A = fil_A_blocks[idx]

# 3D matrix counting how many times each voxel is crossed by rays
cross_matrix = np.zeros((Nlon, Nlat, Nz))

# Count voxel crossings by scanning the system matrix A
for i in range(A.shape[0]):          # Loop over equations (rays)
    for j in range(A.shape[1]):      # Loop over voxels (unknowns)
        if A[i, j] > 0:              # The ray crosses voxel j
            x1, x2, x3 = decode_id(original_idx[idx][j])
            cross_matrix[x1, x2, x3] += 1


# Extract grid dimensions
Nlon, Nlat, Nz = cross_matrix.shape

# Lists for 3D scatter plot coordinates and colors
x_vals, y_vals, z_vals, colors = [], [], [], []


# Populate scatter plot data only for voxels actually crossed by rays
for x1 in range(Nlon):
    for x2 in range(Nlat):
        for x3 in range(Nz):
            if cross_matrix[x1, x2, x3] > 0:
                x_vals.append(x1)
                y_vals.append(x2)
                z_vals.append(x3)
                colors.append(cross_matrix[x1, x2, x3])  # Number of crossings


# Create the 3D figure
fig2 = go.Figure()

# Add 3D scatter plot of voxel crossings
fig2.add_trace(go.Scatter3d(
    x=x_vals,
    y=y_vals,
    z=z_vals,
    mode='markers',
    marker=dict(
        size=7,
        color=colors,                 # Color proportional to crossing count
        colorscale='Plasma',          # Colormap
        colorbar=dict(title='Number of crossings'),
        opacity=0.8
    )
))

# Configure axes labels and layout
fig2.update_layout(
    scene=dict(
        xaxis=dict(title='Longitude [°]'),
        yaxis=dict(title='Latitude [°]'),
        zaxis=dict(title='Altitude [km]')
    ),
    title="3D visualization of atmospheric voxel crossings",
    width=900,
    height=800
)


**CYCLES FOR STEC COLLECTION FOR EVERY GROUND SEGMENT CONFIGURATION**

In [24]:
print(lat_min, lat_max, lon_min, lon_max)
print(f'dimensioni griglia voxels:{Nlon, Nlat, Nz}')

num_vertical_steps = int((Hion-h)/h)
tim = time.time()

grid_min = np.array([lat_min, lon_min, Re+h])

num_vertical_steps = int((Hion-h)/h)
num_steps = 40 #num_vertical_steps * num_oriz_steps 

cosalpha_max = np.cos(alpha_max * np.pi / 180)
print('################', cosalpha_max)


TOTAL_COEFF_MATRIX = []
TOTAL_ISL_COEFF_MATRIX = []
TOTAL_REGULARIZATION_MATRIX= []
TOTAL_STEC_VECTORS = []
TOTAL_ISL_STEC_VECTORS = []
TOTAL_DICTIONARIES = []
TOTAL_INVERS_DICTIONARIES = []

D = 20

hours = 24 

lat_grid = np.arange(lat_min, lat_max, r)
lon_grid = np.arange(lon_min, lon_max, r)
alt_grid = np.arange(Re + (2*h), Re + Hion + h, h)
grid_min =  np.array([lat_min, lon_min, Re+h])
num_steps = 40
time_stv = 1

Aggregation_times = np.array(np.array([0, 30, 40, 50, 60, 75, 90]) *60/dt + 0.5, dtype=int) + instant_dt


#Nstations= [64]
d = 2.5
print('precisione griglia stazioni:', d)
P0=[]
D=20
        
for d0 in np.arange(d/2, int(D), d):
    for d1 in np.arange(d/2, int(D), d):
        P0.append([lat_min + d0 + D/2, lon_min + D/2 + d1])
        
print(P0)  
for i in range(len(P0)):
    P0[i] = np.array(cartesian(P0[i], Re))
print('Numero di stazioni terrestri poste:', len(P0), 'su una griglia di passo:', d, 'in una regione',D*2,'x',D*2, 'in un cono', Dln,'x',Dlt)


STEC_isl_pt = []
total_matrix_isl_pt = []
total_length_matrix_isl_pt =[]
STEC_pt = []
total_matrix_pt = []
total_length_matrix_pt =[]

# CONTO I VERTICAL LINKS
for m, mes_min in enumerate(Aggregation_times[1:]):
    for u, estec in enumerate(list_estec):
        #for e in range(len(estec)): 
        
        print()
        print()
        print('###############################################################')
        print()
        print('I_iono:', Iiono, 'per frequency L5', 'time window block:', mes_min)
        print()
        
        #

        # CONTO I VERTICAL LINKS

        for t in range(Aggregation_times[m], Aggregation_times[m+1]):
            
            if t%10 == 0:
                print(f"Iterazione t:{t - instant_dt} / {Aggregation_times[m+1] - instant_dt}")
                
            satelliti_visti=0
            
            STEC = []
            total_matrix = []
            total_length_matrix = []
            
            if t%time_stv==0: #per evitare un'eccessiva dipendenza tra le equazioni, ne considero una ogni 2
                
                for id_pos, pos0 in enumerate(P0):
                
                    for id_sat, satellite in enumerate(satellite_names):
                    
                        pos1 = np.array(df.loc[t, satellite])
                        cosalpha = vis_alpha(pos1, pos0)
                        l=-1
                        
                        if (cosalpha > cosalpha_max):
                        
                            voxels_vert_mx = - np.ones(int(num_steps), dtype=int)
                            lengths_vert_mx = - np.ones(int(num_steps))
                            
                            satelliti_visti += 1
                            xi1, yi1, zi1 = intersection(pos1, pos0, R=Re+h)
                            xi2, yi2, zi2 = intersection(pos1, pos0, R=Re+Hion)
                            P1 = np.array([xi1, yi1, zi1])
                            P2 = np.array([xi2, yi2, zi2])
            
                            
                            stec_vert = ray_error_vertical_voxel_intersections(P1, P2, Nlon, Nlat, Nz, r, h, grid_min, density, voxels_vert_mx, lengths_vert_mx, estec)
            
                            voxels_vert_mx[-2] = id_pos
                            lengths_vert_mx[-2] = id_pos
                            
                            voxels_vert_mx[-1] = id_sat
                            lengths_vert_mx[-1] = id_sat
                            
                            total_matrix.append(voxels_vert_mx)
                            total_length_matrix.append(lengths_vert_mx)
                            STEC.append(stec_vert)
                            
                 
            STEC_pt.append(STEC)
            total_matrix_pt.append(total_matrix)
            total_length_matrix_pt.append(total_length_matrix)
    
    
                
        print('tot istanti temporali', len(total_matrix_pt))
        
        A_blocks, L_blocks, b_blocks = block_build(total_matrix_pt, total_length_matrix_pt, STEC_pt, mes_min, 1, 0)
        
        fil_A_blocks = []
        fil_L_blocks = []
        non_zero_columns = []
        
        for i in range(len(A_blocks)):
            non_zero_columns.append(np.any(A_blocks[i] != 0, axis=0))
            filtered_A = A_blocks[i][:, non_zero_columns[-1]]
            filtered_L = L_blocks[i][non_zero_columns[-1][:-1]][:, non_zero_columns[-1]]
            fil_A_blocks.append(filtered_A)
            fil_L_blocks.append(filtered_L)
            print(i, 'A_block', 'old shape:', A_blocks[i].shape, 'new shape:', filtered_A.shape)
            #print(i, 'L_block', 'old shape:', L_blocks[i].shape, 'new shape:', filtered_L.shape)
        
        original_idx = []
        original_idx_inv = []
    
        for k in range(len(non_zero_columns)):
            original_idx.append({j: i for j, i in enumerate([i for i in range(len(non_zero_columns[k])) if non_zero_columns[k][i]])})    
            original_idx_inv.append({i: j for j, i in enumerate([i for i in range(len(non_zero_columns[k])) if non_zero_columns[k][i]])})
    
        time_sto=1    
        num_steps = 80            
        matrix_idx = 0
        
        print('horizzontal merging:')

        time_step = int(mes_min/ time_sto)
        print(time_step)
        
        for t in range(Aggregation_times[m], Aggregation_times[m+1]):
            
            STEC = []
            total_matrix = []
            total_length_matrix =[]
        
            pos_s = util_isl[util_isl['time'] == t][['ri1', 'ri2']].reset_index()
            
            for row in range(len(pos_s)):
                voxels_oriz_mx = - np.ones(int(num_steps), dtype=int)
                lengths_oriz_mx = - np.ones(int(num_steps))
                
                P1 = np.array(pos_s.loc[row, 'ri1'])
                P2 = np.array(pos_s.loc[row, 'ri2'])
                stec_oriz = ray_error_orizontal_voxel_intersections(P1, P2, Nlon, Nlat, Nz, r, h, grid_min, density, voxels_oriz_mx, lengths_oriz_mx, matrix_idx, original_idx_inv, estec*isl_iono_reduction)
                
                if voxels_oriz_mx[0]>-0.5:
                    total_matrix.append(voxels_oriz_mx)
                    total_length_matrix.append(lengths_oriz_mx)
                    STEC.append(stec_oriz)
                    
            STEC_isl_pt.append(STEC)
            total_matrix_isl_pt.append(total_matrix)
            total_length_matrix_isl_pt.append(total_length_matrix)

        Aisl_blocks, Lisl_blocks, bisl_blocks = block_build(total_matrix_isl_pt, total_length_matrix_isl_pt, STEC_isl_pt, mes_min, 1, 0)
    
        total_matrix_hv_pt = total_equations(total_matrix_pt, total_matrix_isl_pt, 1, 1, len(total_matrix_pt))
        total_length_matrix_hv_pt = total_equations(total_length_matrix_pt, total_length_matrix_isl_pt, 1, 1, len(total_matrix_pt))
        STEC_hv_pt = total_b(STEC_pt, STEC_isl_pt, 1, 1, len(total_matrix_pt))
                    
        Ahv_blocks, Lhv_blocks, bhv_blocks = block_build(total_matrix_hv_pt, total_length_matrix_hv_pt, STEC_hv_pt, mes_min, 1, 0)
        
        fil_Ahv_blocks = []
    
        for i in range(len(A_blocks)):
            non_zero_columns_hv = np.any(Ahv_blocks[i] != 0, axis=0)
            filtered_A = Ahv_blocks[i][:, non_zero_columns_hv]
            fil_Ahv_blocks.append(filtered_A)
            print(i, 'A_block', 'old shape:', Ahv_blocks[i].shape, 'new shape:', filtered_A.shape)
            
        for i in range(len(Ahv_blocks)):
            if len(Ahv_blocks[i]) - len(A_blocks[i]) != len(Aisl_blocks[i]):
                print(f'allocazione sbagliata dell equazione orizzontale nel blocco {i} esimo')

        TOTAL_COEFF_MATRIX.append(fil_A_blocks[0])
        TOTAL_ISL_COEFF_MATRIX.append(fil_Ahv_blocks[0])
        
        TOTAL_REGULARIZATION_MATRIX.append(fil_L_blocks[0])
        
        TOTAL_STEC_VECTORS.append(b_blocks[0])
        TOTAL_ISL_STEC_VECTORS.append(bhv_blocks[0])
        
        TOTAL_DICTIONARIES.append(original_idx[0])
        TOTAL_INVERS_DICTIONARIES.append(original_idx_inv[0])


30 70 -10 30
dimensioni griglia voxels:(20, 20, 19)
################ 0.6427876096865394
precisione griglia stazioni: 2.5
[[41.25, 1.25], [41.25, 3.75], [41.25, 6.25], [41.25, 8.75], [41.25, 11.25], [41.25, 13.75], [41.25, 16.25], [41.25, 18.75], [43.75, 1.25], [43.75, 3.75], [43.75, 6.25], [43.75, 8.75], [43.75, 11.25], [43.75, 13.75], [43.75, 16.25], [43.75, 18.75], [46.25, 1.25], [46.25, 3.75], [46.25, 6.25], [46.25, 8.75], [46.25, 11.25], [46.25, 13.75], [46.25, 16.25], [46.25, 18.75], [48.75, 1.25], [48.75, 3.75], [48.75, 6.25], [48.75, 8.75], [48.75, 11.25], [48.75, 13.75], [48.75, 16.25], [48.75, 18.75], [51.25, 1.25], [51.25, 3.75], [51.25, 6.25], [51.25, 8.75], [51.25, 11.25], [51.25, 13.75], [51.25, 16.25], [51.25, 18.75], [53.75, 1.25], [53.75, 3.75], [53.75, 6.25], [53.75, 8.75], [53.75, 11.25], [53.75, 13.75], [53.75, 16.25], [53.75, 18.75], [56.25, 1.25], [56.25, 3.75], [56.25, 6.25], [56.25, 8.75], [56.25, 11.25], [56.25, 13.75], [56.25, 16.25], [56.25, 18.75], [58.75, 1.

In [25]:
Nstations = 64

isl_stec_pt_filename = "C:\\Users\\cadan\\OneDrive - Universita' degli Studi di Roma Tor Vergata\\ARTICOLO TESI\\CODICE\\creazione delle matrici di adiacenza\\DATI\\WithNoise_STEC_isl_from_hightKp_IRI_IED" + "_d(grad)=" + str(precision_grad) + "_Nstations=" +str(Nstations)+ "_gaussianError_std="+ str(list_estec) + 'TECU' + '_time_stv=' + str(time_stv) + "_variable_Time_30_to_90_.pkl"

stec_pt_filename = "C:\\Users\\cadan\\OneDrive - Universita' degli Studi di Roma Tor Vergata\\ARTICOLO TESI\\CODICE\\creazione delle matrici di adiacenza\\DATI\\WithNoise_STEC_from_hightKp_IRI_IED" + "_d(grad)=" + str(precision_grad) + "_Nstations=" +str(Nstations) + "_gaussianError_std="+ str(list_estec) + 'TECU' + '_time_stv=' + str(time_stv) + "_variable_Time_30_to_90_.pkl"

Ldense_filename = "C:\\Users\\cadan\\OneDrive - Universita' degli Studi di Roma Tor Vergata\\ARTICOLO TESI\\CODICE\\creazione delle matrici di adiacenza\\DATI\\WithNoise_finite_diff_operator_from_hightKp_IRI_IED" + "_d(grad)=" + str(precision_grad) + "_Nstations=" +str(Nstations)+ "_gaussianError_std="+ str(list_estec) + 'TECU'+ '_time_stv=' + str(time_stv) + "_variable_Time_30_to_90_.pkl"

Adense_filename = "C:\\Users\\cadan\\OneDrive - Universita' degli Studi di Roma Tor Vergata\\ARTICOLO TESI\\CODICE\\creazione delle matrici di adiacenza\\DATI\\WithNoise_coeff_matrix_from_hightKp_IRI_IED" + "_d(grad)=" + str(precision_grad) + "_Nstations=" +str(Nstations)+ "_gaussianError_std="+ str(list_estec) + 'TECU' + '_time_stv=' + str(time_stv) + "_variable_Time_30_to_90_.pkl"

isl_Adense_filename = "C:\\Users\\cadan\\OneDrive - Universita' degli Studi di Roma Tor Vergata\\ARTICOLO TESI\\CODICE\\creazione delle matrici di adiacenza\\DATI\\WithNoise_coeff_matrix_isl_from_hightKp_IRI_IED" + "_d(grad)=" + str(precision_grad) + "_Nstations=" +str(Nstations)+ "_gaussianError_std="+ str(list_estec) + 'TECU' + '_time_stv=' + str(time_stv) + "_variable_Time_30_to_90_.pkl"

dizionario_filename ="C:\\Users\\cadan\\OneDrive - Universita' degli Studi di Roma Tor Vergata\\ARTICOLO TESI\\CODICE\\creazione delle matrici di adiacenza\\DATI\\WithNoise_dictionary_from_hightKp_IRI_IED" + "_d(grad)=" + str(precision_grad) + "_Nstations=" +str(Nstations)+ "_gaussianError_std="+ str(list_estec) + 'TECU' + '_time_stv=' + str(time_stv) + "_variable_Time_30_to_90_.pkl"

inverse_dizionario_filename = "C:\\Users\\cadan\\OneDrive - Universita' degli Studi di Roma Tor Vergata\\ARTICOLO TESI\\CODICE\\creazione delle matrici di adiacenza\\DATI\\WithNoise_inverse_dictionary_from_hightKp_IRI_IED" + "_d(grad)=" + str(precision_grad) + "_Nstations=" +str(Nstations)+ "_gaussianError_std="+ str(list_estec) + 'TECU' + '_time_stv=' + str(time_stv) + "_variable_Time_30_to_90_.pkl"

print(Adense_filename)

with open(inverse_dizionario_filename, 'wb') as file:
    pickle.dump(TOTAL_INVERS_DICTIONARIES, file)
with open(dizionario_filename, 'wb') as file:
    pickle.dump(TOTAL_DICTIONARIES, file)
with open(stec_pt_filename, 'wb') as file:
    pickle.dump(TOTAL_STEC_VECTORS, file)
with open(Adense_filename, 'wb') as file:
    pickle.dump(TOTAL_COEFF_MATRIX, file)
with open(Ldense_filename, 'wb') as file:
    pickle.dump(TOTAL_REGULARIZATION_MATRIX, file)

with open(isl_stec_pt_filename, 'wb') as file:
    pickle.dump(TOTAL_ISL_STEC_VECTORS, file)
with open(isl_Adense_filename, 'wb') as file:
    pickle.dump(TOTAL_ISL_COEFF_MATRIX, file)

C:\Users\cadan\OneDrive - Universita' degli Studi di Roma Tor Vergata\ARTICOLO TESI\CODICE\creazione delle matrici di adiacenza\DATI\WithNoise_coeff_matrix_from_hightKp_IRI_IED_d(grad)=0.4_Nstations=64_gaussianError_std=[1]TECU_time_stv=1_variable_Time_30_to_90_.pkl


In [26]:
for i in range(len(TOTAL_COEFF_MATRIX)):
    print(np.shape(TOTAL_ISL_COEFF_MATRIX[i]), np.shape(TOTAL_COEFF_MATRIX[i]))

(14167, 3728) (14108, 3728)
(18971, 3779) (18901, 3779)
(23397, 3783) (23308, 3783)
(28251, 3804) (28139, 3804)
(35135, 3843) (34991, 3843)
(42019, 3947) (41840, 3947)
